<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Word2Vec" data-toc-modified-id="Word2Vec-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Word2Vec</a></span><ul class="toc-item"><li><span><a href="#How-a-body-is-vectorized" data-toc-modified-id="How-a-body-is-vectorized-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>How a body is vectorized</a></span><ul class="toc-item"><li><span><a href="#Playing-around-with-the-model-and-vectorization-functions" data-toc-modified-id="Playing-around-with-the-model-and-vectorization-functions-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Playing around with the model and vectorization functions</a></span></li></ul></li><li><span><a href="#Concatenating-Question-and-Solution-Vectors" data-toc-modified-id="Concatenating-Question-and-Solution-Vectors-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Concatenating Question and Solution Vectors</a></span></li></ul></li><li><span><a href="#Data-Collection" data-toc-modified-id="Data-Collection-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Collection</a></span><ul class="toc-item"><li><span><a href="#Escaping-Solr-Special-Characters" data-toc-modified-id="Escaping-Solr-Special-Characters-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Escaping Solr Special Characters</a></span></li><li><span><a href="#Organizing-the-instances" data-toc-modified-id="Organizing-the-instances-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Organizing the instances</a></span></li><li><span><a href="#Porting-over-pull_training_data()-from-Ranker" data-toc-modified-id="Porting-over-pull_training_data()-from-Ranker-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Porting over pull_training_data() from Ranker</a></span></li><li><span><a href="#Creating-the-train/test-sets" data-toc-modified-id="Creating-the-train/test-sets-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Creating the train/test sets</a></span></li><li><span><a href="#Creating-Negative-Instances" data-toc-modified-id="Creating-Negative-Instances-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Creating Negative Instances</a></span></li><li><span><a href="#Creating-a-DataFrame" data-toc-modified-id="Creating-a-DataFrame-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Creating a DataFrame</a></span></li><li><span><a href="#Vectorizing-the-bodies" data-toc-modified-id="Vectorizing-the-bodies-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>Vectorizing the bodies</a></span></li><li><span><a href="#Parallelization-with-Dask" data-toc-modified-id="Parallelization-with-Dask-4.8"><span class="toc-item-num">4.8&nbsp;&nbsp;</span>Parallelization with Dask</a></span></li><li><span><a href="#Parallelization-with-PySpark" data-toc-modified-id="Parallelization-with-PySpark-4.9"><span class="toc-item-num">4.9&nbsp;&nbsp;</span>Parallelization with PySpark</a></span><ul class="toc-item"><li><span><a href="#Vectorizing-the-dataframes" data-toc-modified-id="Vectorizing-the-dataframes-4.9.1"><span class="toc-item-num">4.9.1&nbsp;&nbsp;</span>Vectorizing the dataframes</a></span></li></ul></li><li><span><a href="#Adding-vectors-to-the-dataframes" data-toc-modified-id="Adding-vectors-to-the-dataframes-4.10"><span class="toc-item-num">4.10&nbsp;&nbsp;</span>Adding vectors to the dataframes</a></span></li><li><span><a href="#Putting-it-all-together-to-create-the-train/test-sets" data-toc-modified-id="Putting-it-all-together-to-create-the-train/test-sets-4.11"><span class="toc-item-num">4.11&nbsp;&nbsp;</span>Putting it all together to create the train/test sets</a></span></li></ul></li><li><span><a href="#Creating-RankerNet" data-toc-modified-id="Creating-RankerNet-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Creating RankerNet</a></span><ul class="toc-item"><li><span><a href="#Multilayer-Perceptron" data-toc-modified-id="Multilayer-Perceptron-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Multilayer Perceptron</a></span></li><li><span><a href="#SKlearn-Pipeline" data-toc-modified-id="SKlearn-Pipeline-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>SKlearn Pipeline</a></span></li><li><span><a href="#Hyperparameter-Optimization" data-toc-modified-id="Hyperparameter-Optimization-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Hyperparameter Optimization</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Evaluation</a></span></li></ul></li></ul></div>

# Overview
This notebook will explore vectorization of our queries and solutions using a pretrained Word2Vec model. Our current production version of Ranker uses linear regression on vectors with around 35 to 45 elements. Word2Vec will increase the dimensionality of our vectors considerably, as each word vector contains 300 elements and our ultimate vecotorization step will include the concatenation of a few vectors. The general plan is as follows:

* Load in a pretrained w2v model that can take a single token and return its vector
* Pull in training data in the form of a list of dicts
    * The pulled queries are positive instances, as they are linked to solutions
* Sort the training data queries by date and split them into a training and test split
* Convert the list of dicts into a dataframe with the columns `question_id`, `solution_id`, and `target`
* Vectorize the question and solution bodies and index them in a dict mapping from `id` -> `vector`
* Add the appropriate vector to each row in the dataframe
* Train a neural network with the input training vectors and `target` values
* Compute the PCT@5 metric for the queries in the test set. 

While this approach works fine for smaller datasets (~5000 queries), once that number increases into the tens of thousands, memory issues and long computation times require some changes. Those big data approaches will be explored in a different notebook to prevent the size and scope of this notebook from getting too big. The ultimate goal of this notebook will be to get a baseline PCT@5 score using a dataset of 5000 queries, which is the same amount of queries currently used with Ranker.

# Imports
We'll start by using `gensim` and Google's pretrained Word2Vec model, which has already been downloaded and saved on this machine.

In [1]:
import gensim
import numpy as np

from nltk import download, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

# Word2Vec

We will be using Google's pretrained Word2Vec model and the `gensim` package to implement the model. A handful of functions will be created so that a question-solution pair can be vectorized into a single vector which will be used as the input to the neural network. 

## How a body is vectorized
This word2vec model has a 300-dimensional vector for over 3,000,000 words. In order to get a single vector representation for a body of words, we will find the average vector of all of the words in the body that exist in the model and then concatenate that with the element-wise maximum of the vectors of all of the words in the body. This results in a 600-dimensional vector. 

In [3]:
def vectorize_words(words, w2v):
    """Create a list of w2v vectors containing the w2v vector for each word in the given list of words"""
    return [w2v[word] for word in words if word in w2v]


def vectorize_body(body, w2v):
    """Create a 600-dimensional vector for the given body of text and w2v model"""
    try:
        word_vectors = vectorize_words(word_tokenize(body), w2v)
        average_vector = np.mean(word_vectors, axis=0)
        maximum_vector = np.maximum.reduce(word_vectors)
        return np.concatenate([average_vector, maximum_vector])
    except ValueError:
        return None


def vector_similarity(vector1, vector2):
    """Return the cosine similarity between two vectors"""
    return cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))[0][0]

### Playing around with the model and vectorization functions

In [4]:
vectorize_body("Hello there, how are you?", model).size

600

In [5]:
bodies = ["Hello there, how are you?", 
          "Hi buddy, what's up?", 
          "apples, oranges, bananas", 
          "mangos, strawberries, pears"]
vectors = {body: vectorize_body(body, model) for body in bodies}

In [6]:
from itertools import combinations

pair_scores = {}
for body_1, body_2 in combinations(bodies, 2):
    pair_scores[vector_similarity(vectors[body_1], 
                                  vectors[body_2])] = f"'{body_1}' - '{body_2}'"

sorted(pair_scores.items(), reverse=True)

[(0.7927833, "'apples, oranges, bananas' - 'mangos, strawberries, pears'"),
 (0.7504788, "'Hello there, how are you?' - 'Hi buddy, what's up?'"),
 (0.33531186, "'Hello there, how are you?' - 'mangos, strawberries, pears'"),
 (0.32507744, "'Hello there, how are you?' - 'apples, oranges, bananas'"),
 (0.29591754, "'Hi buddy, what's up?' - 'apples, oranges, bananas'"),
 (0.27692673, "'Hi buddy, what's up?' - 'mangos, strawberries, pears'")]

In [7]:
model.most_similar(positive=["Chicago"], topn=10)

/opt/conda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Windy_City', 0.7112736105918884),
 ('Milwaukee', 0.659806489944458),
 ('St._Louis', 0.6426857709884644),
 ('Peoria', 0.6366877555847168),
 ('Naperville', 0.63065505027771),
 ('Evanston', 0.6249232292175293),
 ('Rockford', 0.6170403957366943),
 ('Minneapolis', 0.611090898513794),
 ('Schaumburg', 0.6032407283782959),
 ('Joliet', 0.6019251346588135)]

In [8]:
model.most_similar(positive=["Washington_DC"], topn=10)

[('Arlington_Va.', 0.6351395845413208),
 ('Arlington_VA', 0.6333052515983582),
 ('E_mail_cptime@aol.com', 0.6242501139640808),
 ('Adam_Hoffinger_criminal', 0.6177441477775574),
 ('Alexandria_Va.', 0.6099658012390137),
 ('Alexandria_VA', 0.6096070408821106),
 ('DC', 0.6005620360374451),
 ('Chevy_Chase_Maryland', 0.5873388051986694),
 ('inWashington_DC', 0.5790846347808838),
 ('Washinton_DC', 0.5784515738487244)]

In [9]:
model.most_similar(positive=["Virtual_Machines"], topn=25)

[('virtual_machines', 0.7039488554000854),
 ('Virtual_Machine', 0.6841978430747986),
 ('virtual_machines_VMs', 0.683608889579773),
 ('VMs', 0.6801522374153137),
 ('Hypervisors', 0.6639840006828308),
 ('VMware_ESX_server', 0.6579589247703552),
 ('Virtual_Desktops', 0.6553028225898743),
 ('VMware_ESX', 0.6497218608856201),
 ('VMware_HA', 0.6431201100349426),
 ('virtualized', 0.6400014758110046),
 ('virtualised_environment', 0.638530969619751),
 ('guest_OSs', 0.6369467973709106),
 ('VMware_VMotion', 0.6366690397262573),
 ('Virtual_Servers', 0.63653564453125),
 ('VMWare_ESX', 0.6361702084541321),
 ('MySQL_Database', 0.6353836059570312),
 ('Oracle_RAC', 0.6331586837768555),
 ('Microsoft_Cluster_Server', 0.6320348978042603),
 ('VMware_hypervisors', 0.6307154297828674),
 ('iNSP', 0.6280431747436523),
 ('logical_partitions', 0.6269524097442627),
 ('Virtual_Appliance', 0.625616192817688),
 ('vApp', 0.622918963432312),
 ('ESX_ESXi', 0.6223375797271729),
 ('multipathing', 0.6191391944885254)]

## Concatenating Question and Solution Vectors
Ultimately Ranker takes in a question and solution and returns how likely the solution will resolve the question, so the input to the neural network Ranker will be a vector that represents a question-solution _pair_. To create this input vector, for each solution we need to find all of the questions that have been linked to it. Then we'll vectorize the question, vectorize the solution, and concatenate those two vectors to create a 1200-dimensional vector that represents the question-solution pair.

In [10]:
def concatenate_vectors(question_vector, solution_vector):
    """Create a 1200-dimensional vector for the given question and solution vectors"""
    return np.concatenate([question_vector, solution_vector])

# Data Collection

In order to hold as many aspects constant across the current Ranker and this new neural net Ranker, the data collection functions from Ranker will be ported over to this notebook. The functions are primarily from `/ranker/src/train/service.py` from the `its_ranker` repository. In order to get them to work properly within this notebook, any reference to the `corpus`, `model`, and `collection` had to be replaced with a workaround, usually hardcoded values. 

In [11]:
from collections import defaultdict
from datetime import datetime
from math import ceil

from pysolr import Solr, SolrError
from solr_dsl import Search, Field, Range

DOMAINS = ['VMware vSphere ESXi 5.5', 'VMware vCenter Server 5.5',
           'VMware Update Manager 5.5', 'VMware vCenter Server Appliance 5.5',
           'VMware vSphere Client 5.5']

RESULT_FQ = '+(division_names_Ft:"vSphere 5.5" domain:"VMware vSphere ESXi 5.5" domain:"VMware vCenter Server 5.5" domain:"VMware Update Manager 5.5" domain:"VMware vCenter Server Appliance 5.5" domain:"VMware vSphere Client 5.5" derived__domain_Ft:"VMware vSphere ESXi 5.5" derived__domain_Ft:"VMware vCenter Server 5.5" derived__domain_Ft:"VMware Update Manager 5.5" derived__domain_Ft:"VMware vCenter Server Appliance 5.5" derived__domain_Ft:"VMware vSphere Client 5.5") +(status:EXTERNAL status:INTERNAL status:DRAFT) -status:DELETED -status:PRIVATE_DRAFT'
QUERY_FQ = "+doc_type:question +(division_names_Ft:'vSphere 5.5' domain:'VMware vSphere ESXi 5.5' domain:'VMware vCenter Server 5.5' domain:'VMware Update Manager 5.5' domain:'VMware vCenter Server Appliance 5.5' domain:'VMware vSphere Client 5.5')"
SIMILAR_QUERY_FQ = "+doc_type:question +(division_names_Ft:'vSphere 5.5' domain:'VMware vSphere ESXi 5.5' domain:'VMware vCenter Server 5.5' domain:'VMware Update Manager 5.5' domain:'VMware vCenter Server Appliance 5.5' domain:'VMware vSphere Client 5.5')"

solr = Solr('http://penseur.it9.org:8983/solr/NEW_PRD-vmware-sfkm')

## Escaping Solr Special Characters
An unforseen issue with the proceding functions that query Solr for the most similar solution is that if the query's body contains any of the Solr special characters, there will be severe parsing issues that throw many different errors. These need to be escaped first and that's handled by the following function.

In [44]:
import re


ESCAPE_CHARS_RE = re.compile(r'(?<!\\)(?P<char>[&|+\-!(){}[\] ^"~*?:])')


def solr_escape(value):
    """Escape un-escaped special characters and return escaped value. Inspired by
    https://github.com/Parsely/python-solr/blob/master/pythonsolr/pysolr.py#L181
    but a small change needed to be made to ESCAPE_CHARS_RE (adding a space)
    """
    return ESCAPE_CHARS_RE.sub(r'\\\g<char>', value)

## Organizing the instances
A `pandas` DataFrame will be used to keep track of all of the instances. When pulling the data, the saved fields will be `question_id`, `solution_id`, `target`, and `question_date`. Additionally, a dictionary that maps a document id to a document body will be created so that the document bodies can be quickly retrieved for vectorizing. 


In [45]:
import pandas as pd

## Porting over pull_training_data() from Ranker
The following cell contains the ported code from `its_ranker`. Some progress bars were also added to nicely show the progress of the download. 

In [14]:
from collections import Counter

from ipywidgets import IntProgress
from IPython.display import display
import langid
import pickle


def get_solr_docs(query, fq=[], sort='',batch_size="100", fl="*", limit=None):
    cursorMark = "*"
    count = 0
    while (True):
        results = solr.search(query,
                              fq=fq,
                              cursorMark=cursorMark,
                              fl=fl,
                              sort=sort,
                              rows=batch_size)

        for result in results:
            if 'date' in result:
                try:
                    result['date'] = parser.parse(result['date'])
                except Exception:
                    pass
            yield result

            count += 1
            if limit and count >= limit:
                return
        if cursorMark == results.nextCursorMark:
            return
        cursorMark = results.nextCursorMark
        
        
def get_number_of_result_docs(doc_type, filter_query, fl="*", batch_size="100", limit=None):
    return solr.search("*:*",
                       fq=['doc_type:{}'.format(doc_type),
                           filter_query],
                       fl=fl,
                       sort='date desc,id asc',
                       rows=batch_size).hits


def get_result_docs(doc_type, filter_query, fl="*", batch_size="100", limit=None):
    """Returns docs sorted by date in descending order"""
    return get_solr_docs("*:*",
                         fq=['doc_type:{}'.format(doc_type),
                             filter_query],
                         sort='date desc,id asc',
                         batch_size=batch_size,
                         fl=fl,
                         limit=limit)


def get_number_of_queries(query_fq,
                          with_links_to_field=None,
                          fl="*",
                          batch_size="100",
                          sort='date desc,id asc',
                          limit=None,
                          language_code=None):
    fq = [query_fq]
    if with_links_to_field:
        fq.append("{}:[* TO *]".format(with_links_to_field))
    if language_code:
        clause = "languages:{}".format(language_code)
        fq.append(clause)
        print("Using language code, adding clause: {}".format(clause))

    return solr.search("*:*",
                       fl=fl,
                       fq=fq,
                       sort=sort,
                       batch_size=batch_size,
                       limit=limit).hits


def get_queries(query_fq,
                with_links_to_field=None,
                fl="*",
                batch_size="100",
                sort='date desc,id asc',
                limit=None,
                language_code=None):
    """Returns cases sorted by date in descending order"""
    fq = [query_fq]
    if with_links_to_field:
        fq.append("{}:[* TO *]".format(with_links_to_field))
    if language_code:
        clause = "languages:{}".format(language_code)
        fq.append(clause)
        print("Using language code, adding clause: {}".format(clause))

    return get_solr_docs("*:*",
                         fl=fl,
                         fq=fq,
                         sort=sort,
                         batch_size=batch_size,
                         limit=limit)

    
def fix_dates(obj_array):
    # Hack to parse dates in query objects
    for obj in obj_array:
        if 'date' in obj:
            try:
                obj['date'] = parser.parse(obj['date'])
            except Exception:
                pass

            
def get_query_text(query):
    stripped_title = (query['title'].strip()
                      if 'title' in query else '')
    stripped_body = (query['body'].strip()
                     if 'body' in query else '')
    text = stripped_title + ' ' + stripped_body
    return text



def pull_training_data(limit=1000):
    number_of_docs = get_number_of_result_docs('solution', RESULT_FQ)
    print(f"Pulling {number_of_docs} docs.")
    progress_bar = IntProgress(min=0, max=number_of_docs, description='Pulling docs:', bar_style='info')
    display(progress_bar)      
    
    docs = []
    for count, doc in enumerate(get_result_docs("solution", RESULT_FQ)):
        if count % 1000 == 0:
            progress_bar.value = count
        docs.append(doc)

    progress_bar.value = number_of_docs
    print("Pulled %d docs" % len(docs))
    assert len(docs) > 0, "Did not find any docs in ITS!  Check configuration."
    fix_dates(docs)
    
    doc_id2body = {row['id']: row["body"] for row in docs}

    # Define different filtering funcs
    def _filter_length(query):
        """Filter by query length"""
        text = get_query_text(query)
        if text == 'N/A N/A' or len(text) < 6:
            return False, "Not long enough"
        return True, ""

    def _filter_date(query):
        """Don't accept queries without dates"""
        if 'date' not in query:
            return False, "No date"
        return True, ""

    def _filter_language(query):
        # Accept only english queries
        text = get_query_text(query)
        lang, score = langid.classify(text)
        if lang != 'en' and score > 0.99:  # langid is a bit inaccurate, demand a high threshold
            return False, "Non-english"

        return True, ""

    links_to_field_name = "links_to_solution"
    doc_ids = set(doc_id2body.keys())
    blacklisted_docs = set()

    def _filter_links(query):
        """Filter links by which documents are actually in the model.  After filtering
           remove queries that no longer have any valid links
        """
        filtered_links = []
        for link in query[links_to_field_name]:
            if link not in doc_ids:
                continue

            if link in blacklisted_docs:
                continue

            filtered_links.append(link)

        if filtered_links:
            query[links_to_field_name] = filtered_links
            return True, ""

        return False, "Linked to docs not included in model"

    filter_funcs = [
        _filter_length,
        _filter_date,
        _filter_language,
        _filter_links
    ]

    number_of_queries = get_number_of_queries(QUERY_FQ, links_to_field_name)
    print(f"Found {number_of_queries} queries. Query limit: {limit}")  
    if not limit:
        limit = number_of_queries
    progress_bar = IntProgress(min=0, max=limit, description='Pulling queries:', bar_style='info')
    display(progress_bar)
          
    question_id2body = {}
    queries = []
    skipped = []
    reasons = Counter()
    for count, query in enumerate(get_queries(QUERY_FQ, links_to_field_name)):
        if count % 1000 == 0:
            progress_bar.value = count
        skip = False
        for func in filter_funcs:
            include, reason = func(query)
            if not include:
                query['SKIP_REASON'] = reason
                reasons.update([reason])
                skipped.append(query)
                skip = True
                break

        if skip:
            continue

        queries.append(query)
        question_id2body[query["id"]] = query["body"]
        if len(queries) > limit - 1:
            print("Reached limit")
            break

    progress_bar.value = limit
    assert len(queries) > 0, "Did not find any queries in Solr!  Check configuration."

    print("Pulled {} queries, {} skipped".format(len(queries), len(skipped)))
    print("Skipped reasons:", reasons)

    fix_dates(queries)

    return queries, doc_id2body, question_id2body, skipped


def save(obj, filename):
    pickle.dump(obj, open(filename, "wb"))
    
    
def load(filename):
    return pickle.load(open(filename, "rb"))

In [17]:
queries, doc_id2body, question_id2body, skipped = pull_training_data(limit=100)

Pulling 3561 docs.


IntProgress(value=0, bar_style='info', description='Pulling docs:', max=3561)

Pulled 3561 docs
Found 113643 queries. Query limit: 100


IntProgress(value=0, bar_style='info', description='Pulling queries:')

Reached limit
Pulled 100 queries, 109 skipped
Skipped reasons: Counter({'Linked to docs not included in model': 102, 'Not long enough': 6, 'Non-english': 1})


## Creating the train/test sets
The train/test split is done on a pivot date to ensure that all of the queries in the test set were asked after the qeuries in the training set. This emulates the production behavior of Ranker; we can't train on questions that haven't been asked yet.

In [18]:
def split_train_test(queries, date_split_proportion):
    """Split data into test and train sets, based upon the data of the input
    queries. The training and test sets will contain only the `max_num_queries`
    most recent queries.

    :param queries: The sample to be split.
    :param date_split_proportion: The fraction of original queries to be placed
        in the training set.
    :type queries: [dict]
    :type date_split_proportion: float
    :rtype: datetime, [dict], [dict]

    """
    queries_by_date = sorted(queries, key=lambda x: x['date'])
    offset = int(len(queries_by_date) * date_split_proportion)
    date_pivot_point = queries_by_date[offset]['date']
    return date_pivot_point, queries_by_date[:offset], queries_by_date[offset:]

In [19]:
date_pivot, train, test = split_train_test(queries, date_split_proportion=0.75)
print("%d queries in train set" % len(train))
print("%d queries in test set" % len(test))
print("Date pivot point:", date_pivot)

75 queries in train set
25 queries in test set
Date pivot point: 2018-10-10T04:54:38Z


## Creating Negative Instances
Given a list of queries, the next step is to create the negative instances. This is done using a Solr query that retrieves the 200 most similar solutions to a question and all of those solutions are combined with the question to be a negative instance if they aren't already linked. 

In [20]:
def get_top_n_unlinked_solution_ids_from_solr(question, n):
    query = question["body"]
    response = solr.search(solr_escape(query.replace("\n", " ")),
                           fq=RESULT_FQ,
                           fl='id, score',
                           rows=n)
    for doc in response.docs:
        if "links_to_solution" in question:
            if doc["id"] not in question["links_to_solution"]:
                yield doc["id"]

## Creating a DataFrame
The next step is to convert the raw data into a DataFrame that contains the `question_id`, `solution_id`, and `target`. The question date will not be used; the input to the function that creates the DataFrame will ignore the date. This means we'll call this function once on the train set queries and again on the test set queries to create the train DataFrame and test DataFrame.

In [21]:
def create_instance(question_id, solution_id, target):
    return {"question_id": question_id,
            "solution_id": solution_id,
            "target": target}

def create_dataframe(queries, n):
    progress_bar = IntProgress(min=0, max=len(queries), description='Adding queries:', bar_style='info')
    display(progress_bar)

    instances = []
    for count, query in enumerate(queries):
        if count % 1000 == 0:
            progress_bar.value = count
        
        # create positive instances
        for solution_id in query["links_to_solution"]:
            instances.append(create_instance(query["id"], solution_id, target=1))

        # find and create negative instances
        try:
            for solution_id in get_top_n_unlinked_solution_ids_from_solr(query, n):
                instances.append(create_instance(query["id"], solution_id, target=0))
        except SolrError:
            pass
    progress_bar.value = len(queries)
    return pd.DataFrame(instances)

In [22]:
test_df = create_dataframe(test, 3)
test_df.head(10)

IntProgress(value=0, bar_style='info', description='Adding queries:', max=25)

,question_id,solution_id,target
0,18bc222b-c8f3-5940-b622-32d3bbbae8bc,09db7ca3-5e76-50c3-9f34-c20cbdc1cc98,1
1,18bc222b-c8f3-5940-b622-32d3bbbae8bc,901c1086-31d6-54bd-8c03-6be55205b68a,0
2,18bc222b-c8f3-5940-b622-32d3bbbae8bc,154a6f02-3fb8-5855-8427-9c96c3200b8f,0
3,18bc222b-c8f3-5940-b622-32d3bbbae8bc,bfda5d3e-5d86-5afc-b2ad-72233d48c172,0
4,1ca65309-a397-509d-9168-d4e0a3705a8a,4003573f-9485-5f3b-a4e7-b545637eb886,1
5,1ca65309-a397-509d-9168-d4e0a3705a8a,e1189788-a21c-5d65-92df-501b7bbf6300,0
6,1ca65309-a397-509d-9168-d4e0a3705a8a,e8d4fb3a-ba6a-5258-ae6f-0c771305b048,0
7,1660457c-b4e4-5ac8-ba5c-b15f04efc208,a2faa2cd-c948-5a86-a066-bbea680dd41a,1
8,1660457c-b4e4-5ac8-ba5c-b15f04efc208,d6918260-fcac-5876-b915-efd148677fd6,0
9,1660457c-b4e4-5ac8-ba5c-b15f04efc208,00719985-29f1-589b-9960-5dc1ba5ec761,0


## Vectorizing the bodies
The following function will take in the id -> body dictionaries and return an id -> vector dictionary. These indexed vectors will greatly speed up the vectorization of each question-solution pair, since the question and solution are vectorized independently and each question appears in ~200 instances so we wouldn't want to vectorize that multiple times. With the id -> vector dict, we can iterate across the rows in the instance dataframe and create the instance vector by looking up the question vector and solution vector and then concatenating them.  

In [23]:
def get_id2vector(doc_id2body, question_id2body, w2v):
    number_of_vectors = len(doc_id2body) + len(question_id2body)
    progress_bar = IntProgress(min=0, max=(number_of_vectors - 1), description='Vectorizing:', bar_style='info')
    display(progress_bar)
    
    skipped_questions = {}
    skipped_docs = {}
    id2vector = {}
    for count, (doc_id, body) in enumerate(doc_id2body.items()):
        if count % 1000 == 0:
            progress_bar.value = count
        vector = vectorize_body(body, w2v)
        if vector is not None:
            id2vector[doc_id] = vector
        else:
            skipped_docs[doc_id] = body
    
    prev_count = count
    for count, (question_id, body) in enumerate(question_id2body.items()):
        if count % 1000 == 0:
            progress_bar.value = prev_count + count
        vector = vectorize_body(body, w2v)
        if vector is not None:
            id2vector[question_id] = vector
        else:
            skipped_questions[question_id] = body
    progress_bar.value = number_of_vectors - 1
    return id2vector, skipped_docs, skipped_questions

In [24]:
id2vector, skipped_docs, skipped_questions = get_id2vector(doc_id2body, question_id2body, model)

IntProgress(value=0, bar_style='info', description='Vectorizing:', max=3660)

In [25]:
len(id2vector)

3661

In [26]:
id2vector[queries[0]["id"]]

array([ 4.68046404e-02,  4.80421400e-03,  1.80859044e-02,  3.77476998e-02,
       -9.31000188e-02,  5.01505546e-02, -1.59560307e-03, -2.32217573e-02,
        9.93321761e-02,  5.92856929e-02, -9.23220292e-02, -5.46396039e-02,
       -2.11357549e-02, -1.27525330e-02, -3.51381823e-02,  6.18998222e-02,
        3.58242467e-02,  6.69381917e-02, -2.58411840e-02, -8.98434296e-02,
        6.25610352e-04,  4.12623100e-02, -4.25516777e-02,  1.04795033e-02,
       -8.27323087e-03, -1.73613224e-02, -1.08453065e-01,  1.26775533e-01,
        4.71773148e-02, -4.98174019e-02, -1.11055113e-02, -1.57212950e-02,
       -2.24246979e-02, -3.99213359e-02, -6.61082789e-02, -4.97627258e-02,
       -2.41673794e-02, -1.40694510e-02,  2.14653015e-02,  2.31734384e-02,
        3.11686192e-02,  4.03832346e-02,  1.04281954e-01,  1.29183661e-02,
       -3.15576121e-02, -7.05483779e-02, -3.38601023e-02, -3.10077667e-02,
       -7.04659335e-03, -8.36171024e-03, -6.94794133e-02, -4.17166799e-02,
        2.12425664e-02, -

## Parallelization with Dask
We'll want to utilize parallel computing wherever possible, as the large dataset results in compute times of over several hours for both the data collection and vectorization steps. The [`dask`](https://dask.org/) library will be used. This section is still a WIP. When used with the full dataset of ~70,000 queries, there were some issues that prevented it from working. I'll leave this section in the notebook for possible future reference, but you can skip to the __________ section. 

In [15]:
# !{"dask-scheduler &"}

In [1]:
from dask.distributed import Client, LocalCluster, progress
import dask.dataframe as dd

# if client:
#     client.close()
    
client = Client(n_workers=2, threads_per_worker=8, address='tcp://172.17.0.3:8786')

In [2]:
client.restart()

Client Scheduler: tcp://172.17.0.3:8786 Dashboard: http://172.17.0.3:8787/status,Cluster Workers: 2 Cores: 16 Memory: 128.77 GB


In [27]:
def vectorize_row(row):
    return concatenate_vectors(id2vector[row.question_id], id2vector[row.solution_id])

In [15]:
small_test_df = test_df

test_dd = dd.from_pandas(small_test_df, npartitions=25)
test_dd["vector"] = test_dd.apply(vectorize_row, axis=1, meta=pd.Series(dtype='float32', name='vector'))
# test_dd = test_dd.persist()

In [16]:
test_dd = test_dd.persist()

In [21]:
# rdd.map(lambda row: vectorize_body(row[1], spark_w2v_dict.value)).collect()


[array([0.052948  , 0.03463123, 0.01406804, ..., 0.36523438, 0.40820312,
        0.39257812], dtype=float32),
 array([0.052948  , 0.03463123, 0.01406804, ..., 0.35742188, 0.51171875,
        0.5234375 ], dtype=float32),
 array([0.052948  , 0.03463123, 0.01406804, ..., 0.3046875 , 0.40820312,
        0.41601562], dtype=float32),
 array([0.052948  , 0.03463123, 0.01406804, ..., 0.59375   , 0.52734375,
        0.6640625 ], dtype=float32),
 array([0.052948  , 0.03463123, 0.01406804, ..., 0.38867188, 0.3203125 ,
        0.578125  ], dtype=float32)]

In [91]:
test_dd["vector"] = test_dd.apply(vectorize_row, axis=1, meta=pd.Series(dtype='float32', name='vector'))

In [93]:
vector_df = test_dd.compute()
vector_df.head(10)

,question_id,solution_id,target,vector
0,e45509de-a086-5ba0-a268-9ec9f561736a,23b97fc0-e2a3-5aa0-8cc2-7f579ecffe78,1,"[0.084625244, 0.021258883, 0.007941352, -0.010..."
1,e45509de-a086-5ba0-a268-9ec9f561736a,3d13938b-bedc-5658-8fcd-89ad94a8687c,0,"[0.084625244, 0.021258883, 0.007941352, -0.010..."
2,e45509de-a086-5ba0-a268-9ec9f561736a,77421b0d-6b7b-5223-894f-7c149a2a11ad,0,"[0.084625244, 0.021258883, 0.007941352, -0.010..."
3,1ca9a1c4-3c70-5056-8f04-9291728455b9,9977be47-3477-55c8-8cf0-5ee5542aefbe,1,"[-0.02550987, 0.03704721, 0.101571545, 0.09271..."
4,1ca9a1c4-3c70-5056-8f04-9291728455b9,e0d48678-b127-5b7b-a26b-b84a027fec64,0,"[-0.02550987, 0.03704721, 0.101571545, 0.09271..."
5,1ca9a1c4-3c70-5056-8f04-9291728455b9,25fe17bc-22b3-52cf-91da-752f0ee55d7f,0,"[-0.02550987, 0.03704721, 0.101571545, 0.09271..."
6,1ca9a1c4-3c70-5056-8f04-9291728455b9,187362a9-1305-5079-ad25-ff9749ce0586,0,"[-0.02550987, 0.03704721, 0.101571545, 0.09271..."
7,f528ca88-1c71-5e11-b762-eb9ae3c626b2,1f128b52-b0a0-5fe6-86a1-71ab8a0f02a5,1,"[0.054147575, -0.10364708, 0.049870417, 0.0237..."
8,f528ca88-1c71-5e11-b762-eb9ae3c626b2,bf09fd32-88f1-59a3-831b-5f2a1b8bb880,1,"[0.054147575, -0.10364708, 0.049870417, 0.0237..."
9,f528ca88-1c71-5e11-b762-eb9ae3c626b2,ccba13ac-0bf1-5aa3-a900-3d0937d3e6cf,0,"[0.054147575, -0.10364708, 0.049870417, 0.0237..."


## Parallelization with PySpark
Similar to the preceding Dask section, this is more prototyping code and can skipped.

In [31]:
from pyspark import SparkContext, SparkConf
# sc.stop()

conf = SparkConf().setAppName("App")
conf = (conf.setMaster('local[8]')
        .set('spark.driver.memory', '15G')
        .set('spark.driver.maxResultSize', '0')
        .set('spark.driver.host', 'localhost'))
sc = SparkContext(conf=conf)
# sc = SparkContext()
# import os

# memory = '4g'
# pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
# os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

# sc = SparkContext(appName="Pi")

In [11]:
spark_id2vector = sc.broadcast(id2vector)

In [33]:
sc.uiWebUrl

AttributeError: 'SparkSession' object has no attribute 'uiWebUrl'

In [61]:
spark.stop()

In [58]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

conf = SparkConf().setAppName("RankerNet2")
conf = (conf.setMaster('local[8]')
        .set('spark.driver.memory', '150G')
        .set('spark.driver.maxResultSize', '0')
        .set('spark.driver.host', 'localhost'))

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [50]:
from pyspark.sql.types import StructType, StringType, IntegerType, StructField, ArrayType, FloatType
from pyspark.sql.functions import udf

vectorize_udf = udf(lambda qid, sid: np.concatenate([id2vector[qid], id2vector[sid]]).tolist(), ArrayType(FloatType()))

my_schema = StructType([StructField("question_id", StringType(), False),
                        StructField("solution_id", StringType(), False),
                        StructField("target", IntegerType(), False)])

In [40]:
small_test_df = test_df[0:1000]
df = spark.createDataFrame(small_test_df, schema=my_schema)
df2 = df.withColumn("vector", vectorize_udf(df.question_id, df.solution_id))

In [44]:
vector_list = df2.select("vector").collect()
vector_array = [row.vector for row in vector_list]

### Vectorizing the dataframes

In [59]:
test_vector_df = spark.createDataFrame(test_df, schema=my_schema)
test_vector_df = test_vector_df.withColumn("vector", vectorize_udf(test_vector_df.question_id, 
                                                                   test_vector_df.solution_id))

In [60]:
train_vector_df = spark.createDataFrame(train_df, schema=my_schema)
train_vector_df = train_vector_df.withColumn("vector", vectorize_udf(train_vector_df.question_id, 
                                                                     train_vector_df.solution_id))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.readRDDFromFile.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.api.python.PythonRDD$.readRDDFromInputStream(PythonRDD.scala:188)
	at org.apache.spark.api.python.PythonRDD$.readRDDFromFile(PythonRDD.scala:175)
	at org.apache.spark.api.python.PythonRDD.readRDDFromFile(PythonRDD.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.

## Adding vectors to the dataframes
Now that the bodies have been vectorized and indexed, the next function will take a DataFrame of question_id-solution_id pairs and add the vector as a new column. 

In [46]:
def prune(queries, skipped_questions):
    skipped_question_ids = set(skipped_questions.keys())
    return [query for query in queries if query["id"] not in skipped_question_ids]
    
def add_vectors(dataframe, id2vector):
    progress_bar = IntProgress(min=0, max=(len(dataframe) - 1), description='Adding vectors:', bar_style='info')
    display(progress_bar)
    print(f"Adding {len(dataframe)} vectors.")
    vectors = []
    for count, (index, row) in enumerate(dataframe.iterrows()):
        if count % 10000 == 0:
            progress_bar.value = count
        vectors.append(concatenate_vectors(id2vector[row["question_id"]], id2vector[row["solution_id"]]))
            
            
    progress_bar.value = len(dataframe) - 1
    dataframe["vector"] = pd.Series(vectors, index=dataframe.index)
    return dataframe    

In [28]:
vector_df = add_vectors(test_df, id2vector)

IntProgress(value=0, bar_style='info', description='Adding vectors:', max=107)

Adding 108 vectors.


In [29]:
vector_df.head(10)

,question_id,solution_id,target,vector
0,e45509de-a086-5ba0-a268-9ec9f561736a,23b97fc0-e2a3-5aa0-8cc2-7f579ecffe78,1,"[0.084625244, 0.021258883, 0.007941352, -0.010..."
1,e45509de-a086-5ba0-a268-9ec9f561736a,3d13938b-bedc-5658-8fcd-89ad94a8687c,0,"[0.084625244, 0.021258883, 0.007941352, -0.010..."
2,e45509de-a086-5ba0-a268-9ec9f561736a,77421b0d-6b7b-5223-894f-7c149a2a11ad,0,"[0.084625244, 0.021258883, 0.007941352, -0.010..."
3,1ca9a1c4-3c70-5056-8f04-9291728455b9,9977be47-3477-55c8-8cf0-5ee5542aefbe,1,"[-0.02550987, 0.03704721, 0.101571545, 0.09271..."
4,1ca9a1c4-3c70-5056-8f04-9291728455b9,e0d48678-b127-5b7b-a26b-b84a027fec64,0,"[-0.02550987, 0.03704721, 0.101571545, 0.09271..."
5,1ca9a1c4-3c70-5056-8f04-9291728455b9,25fe17bc-22b3-52cf-91da-752f0ee55d7f,0,"[-0.02550987, 0.03704721, 0.101571545, 0.09271..."
6,1ca9a1c4-3c70-5056-8f04-9291728455b9,187362a9-1305-5079-ad25-ff9749ce0586,0,"[-0.02550987, 0.03704721, 0.101571545, 0.09271..."
7,f528ca88-1c71-5e11-b762-eb9ae3c626b2,1f128b52-b0a0-5fe6-86a1-71ab8a0f02a5,1,"[0.054147575, -0.10364708, 0.049870417, 0.0237..."
8,f528ca88-1c71-5e11-b762-eb9ae3c626b2,bf09fd32-88f1-59a3-831b-5f2a1b8bb880,1,"[0.054147575, -0.10364708, 0.049870417, 0.0237..."
9,f528ca88-1c71-5e11-b762-eb9ae3c626b2,ccba13ac-0bf1-5aa3-a900-3d0937d3e6cf,0,"[0.054147575, -0.10364708, 0.049870417, 0.0237..."


## Putting it all together to create the train/test sets 

In [28]:
queries, doc_id2body, question_id2body, skipped = pull_training_data(limit=5000)

Pulling 3561 docs.


IntProgress(value=0, bar_style='info', description='Pulling docs:', max=3561)

Pulled 3561 docs
Found 113643 queries. Query limit: 5000


IntProgress(value=0, bar_style='info', description='Pulling queries:', max=5000)

Reached limit
Pulled 5000 queries, 6868 skipped
Skipped reasons: Counter({'Linked to docs not included in model': 6639, 'Not long enough': 224, 'Non-english': 5})


In [29]:
id2vector, skipped_docs, skipped_questions = get_id2vector(doc_id2body, question_id2body, model)

IntProgress(value=0, bar_style='info', description='Vectorizing:', max=8560)

/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [38]:
save(id2vector, "id2vector.p")
save(skipped_docs, "skipped_docs.p")
save(skipped_questions, "skipped_questions.p")

In [3]:
id2vector = load("id2vector.p")

In [32]:
pruned_queries = prune(queries, skipped_questions)

In [33]:
date_pivot, train, test = split_train_test(pruned_queries, date_split_proportion=0.75)
print("%d queries in train set" % len(train))
print("%d queries in test set" % len(test))
print("Date pivot point:", date_pivot)

train_df = create_dataframe(train, n=200)
test_df = create_dataframe(test, n=200)

3677 queries in train set
1226 queries in test set
Date pivot point: 2018-08-28T08:13:42Z


IntProgress(value=0, bar_style='info', description='Adding queries:', max=3677)

IntProgress(value=0, bar_style='info', description='Adding queries:', max=1226)

In [41]:
save(train_df, "train_df.p")
save(test_df, "test_df.p")

In [4]:
train_df = load("train_df.p")
test_df = load("test_df.p")

In [34]:
train_vector_df = add_vectors(train_df, id2vector)

IntProgress(value=0, bar_style='info', description='Adding vectors:', max=736642)

Adding 736643 vectors.


In [ ]:
save(train_vector_df, "train_vector_df.p")

In [35]:
test_vector_df = add_vectors(test_df, id2vector)

IntProgress(value=0, bar_style='info', description='Adding vectors:', max=245654)

Adding 245655 vectors.


In [ ]:
save(train_vector_df, "train_vector_df.p")
save(test_vector_df, "test_vector_df.p")

In [21]:
train_vector_df = load("train_vector_df.p")
test_vector_df = load("test_vector_df.p")

# Creating RankerNet
There are a few approaches we can take at this point. For now, I'll stick with `Sklearn` and use the [`MLPClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) class to create a basic neural network. Other library options include `Keras` and/or `Tensorflow`, which will be discussed/explored at a later point.

## Multilayer Perceptron
A [multilayer perceptron](https://scikit-learn.org/stable/modules/neural_networks_supervised.html#multi-layer-perceptron) (MLP) is a class of feedforward neural network that consists of at least three layers (an input layer, a hidden layer, and an output layer) and uses backpropogation for training. It uses an nonlinear activation function and so it can distinguish data that is not linearly separable. 

In [36]:
from sklearn.neural_network import MLPClassifier

## SKlearn Pipeline
I'll use a [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) to implement any preprocessing steps and the model training/testing. Here's the SKlearn [User Guide](https://scikit-learn.org/stable/modules/compose.html#pipeline) for pipelines and [this blog post](https://www.kaggle.com/baghern/a-deep-dive-into-sklearn-pipelines) has a decent example of using `Pipeline`s.

Essentially, I just need to provide a list of steps that need to be performed and then I can call the `fit()`, `transform()`, and `predict()` methods on the `Pipeline` object to do all of the steps in the pipeline. This also allows for straightforward hyperparameter optimization.

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def format_params(params):
    results = {}
    for name, value in params.items():
        if "__" in name:
            key = name.split("__")[1]
        else:
            key = name
        results[key] = value
    return results


def get_pipeline(params={}):
    return Pipeline([
        ('scaler', StandardScaler()),
        ('neural_net', MLPClassifier(**format_params(params)))
    ])

In [38]:
# pipeline = get_pipeline({"hidden_layer_sizes": (300, 150, 50), "activation": 'tanh'})
pipeline = get_pipeline({"hidden_layer_sizes": (300, 150, 50)})
pipeline.fit(train_vector_df["vector"].values.tolist(), train_vector_df["target"].values.tolist())

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('neural_net', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(300, 150, 50), learning_rate='constant',
    ...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])

In [39]:
preds = [prediction[1] for prediction in pipeline.predict_proba(test_vector_df["vector"].values.tolist())]

In [40]:
list(zip(test_vector_df["target"], preds))

[(1, 1.0803807240947396e-08),
 (0, 3.1770454387038627e-15),
 (0, 6.592083104447964e-10),
 (0, 9.402610255405655e-20),
 (0, 3.87812986283893e-07),
 (0, 6.970141502610595e-28),
 (0, 1.438499562391309e-08),
 (0, 1.0323411661835654e-14),
 (0, 1.5525127204273383e-28),
 (0, 5.059440135774606e-26),
 (0, 6.50079514069425e-18),
 (0, 0.0010634755253763803),
 (0, 2.0533869302603653e-16),
 (0, 7.70303458485169e-11),
 (0, 2.5895215889359785e-13),
 (0, 7.217047492493564e-05),
 (0, 2.7866803373842085e-10),
 (0, 7.892135904527992e-11),
 (0, 1.9835815280379971e-13),
 (0, 9.41585440415827e-06),
 (0, 6.987420768738698e-31),
 (0, 4.0524379705122816e-18),
 (0, 1.0019843898835443e-29),
 (0, 3.1052245717932197e-19),
 (0, 1.2297240764538454e-14),
 (0, 5.105512382501639e-06),
 (0, 2.885539906733892e-07),
 (0, 1.8469286047569797e-18),
 (0, 6.0738809368851115e-19),
 (0, 1.1553505393694857e-25),
 (0, 7.407657543177657e-14),
 (0, 2.5739958647981515e-13),
 (0, 1.0079974527174638e-15),
 (0, 0.0029108085589734096),
 

## Hyperparameter Optimization
We can perform a Grid Search on the pipeline to explore different possible hyperparameters. Here's an example:

In [681]:
from sklearn.model_selection import GridSearchCV


param_grid = [
    {'neural_net__hidden_layer_sizes': [(100), (200), (300), (400), (500), (600),
                                        (100, 100), (200, 200), (300, 300), (400, 400), (500, 500), (600, 600),
                                        (100, 100, 100), (200, 200, 200), (300, 300, 300), (400, 400, 400), (500, 500, 500), (600, 600, 600)]}
]

clf = GridSearchCV(pipeline, param_grid)
clf.fit(train_vector_df["vector"].values.tolist(), train_vector_df["target"].values.tolist())

print(clf.best_params_)

{'neural_net__hidden_layer_sizes': (400, 400)}


In [697]:
clf.cv_results_

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([  85.28358436,  137.19048055,  179.32667653,  197.22606365,
         176.17624585,  257.40973266,   86.85834074,  129.53255955,
         161.16800904,  217.34247708,  238.99648921,  309.59553417,
          65.48959605,  111.49512784,  165.51994316,  178.35987473,
         255.22522998,  317.61601734]),
 'std_fit_time': array([ 12.2410212 ,  26.25020271,  38.06406286,  27.07975006,
         36.73464415,  33.5312888 ,  27.49287247,  21.9411497 ,
         35.33171659,  17.28419037,  33.0826622 ,  24.32586396,
          4.23772138,  10.70225734,  14.85757244,   0.19518029,
         13.11577685,  14.29969914]),
 'mean_score_time': array([ 0.86001571,  0.96925465,  1.14586496,  1.20358109,  1.28908491,
         1.36947632,  0.89642302,  1.07871946,  1.26913913,  1.47742756,
         1.68615905,  2.19113302,  0.93604954,  1.22336594,  1.49517131,
         1.7696751 ,  2.0104359 ,  2.61416364]),
 'std_score_time': array([ 0.00389001,  0.05807132,  0.06024758,  0.052859

In [699]:
clf.get_params()

{'cv': None,
 'error_score': 'raise',
 'estimator__memory': None,
 'estimator__steps': [('scaler',
   StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('neural_net',
   MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
          beta_2=0.999, early_stopping=False, epsilon=1e-08,
          hidden_layer_sizes=(300, 300), learning_rate='constant',
          learning_rate_init=0.001, max_iter=200, momentum=0.9,
          nesterovs_momentum=True, power_t=0.5, random_state=None,
          shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
          verbose=False, warm_start=False))],
 'estimator__scaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'estimator__neural_net': MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=(300, 300), learning_rate='constant',
        learning_rate_init=0.001, max_iter=200, moment

In [741]:
best_pipeline = get_pipeline(clf.best_params_)
best_pipeline.fit(train_vector_df["vector"].values.tolist(), train_vector_df["target"].values.tolist())
best_predictions = best_pipeline.predict(test_vector_df["vector"].values.tolist())

In [742]:
r2_score(test_vector_df["target"].values.tolist(), best_predictions)

0.68421985088690807

In [791]:
best_pipeline.predict(test_vector_df["vector"].values.tolist()[:1])

array([ 0.00276197])

## Evaluation
Similar to Ranker, we'll use the PCT@5 metric to evaluate the performance of the neural network. For each of the positive instances in the test set, we'll use the neural network's prediction probability to rank how likely the top 200 solutions are to be matches, and we'll compute the percentage of cases where the top 5 ranked solutions contain at least one linked solution.

In [65]:
from collections import Counter 
from heapq import nlargest

def get_pct5(neural_net, test_vector_df):
    num_vectors_counter = Counter()    
    
    progress_bar = IntProgress(min=0, max=len(test_vector_df), description='Progress:', bar_style='info')
    display(progress_bar)

    visited_questions = set()
    questions = 0
    successes = 0
    for count, (index, row) in enumerate(test_vector_df[test_vector_df["target"] == 1].iterrows()):
        question_id = row["question_id"]
        if question_id not in visited_questions:
            vectors = test_vector_df[test_vector_df["question_id"] == question_id]["vector"].values.tolist()
            num_vectors_counter.update([len(vectors)])
            probas = [prediction[1] for prediction in neural_net.predict_proba(vectors)]
            if 0 in nlargest(5, range(len(probas)), key=lambda idx: probas[idx]):
                successes += 1
            questions += 1
        visited_questions.add(question_id)
        if count+1 % 1000 == 0:
            progress_bar.value = count+1

    progress_bar.value = len(test_vector_df)
    return successes, questions, num_vectors_counter
        

In [66]:
successes, questions, num_vectors_counter = get_pct5(pipeline, test_vector_df)
print(f"{successes}/{questions} = {successes/questions}")

IntProgress(value=0, bar_style='info', description='Progress:', max=245655)

460/1226 = 0.37520391517128876


In [67]:
num_vectors_counter

Counter({200: 565, 201: 566, 202: 73, 204: 4, 203: 15, 2: 1, 65: 1, 215: 1})

In [69]:
len(set(test_vector_df["question_id"]))

1226